United Nations ComTrade
=====

## Bilateral trade data by product

----

*September 3, 2017*<br>
*@bd_econ*

This example retrieves annual data for the trade of a specific product by all countries for which data are available.

[Documentation](https://comtrade.un.org/data/doc/api/) for the UN Comtrade API.

This example uses a list of country codes stored as a csv file. 

In [1]:
import requests
import pandas as pd
import time

# Used to loop over countries 5 at a time.
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

c_codes = pd.read_csv('codes/country_codes.csv').set_index('id')

## Paramaters/ Settings for request

In [2]:
prod_type = 'C'  # Commodity
freq = 'A'       # Annual 
classification = 'HS' # harmonized system
prod = '440710'   # HS 6-digit production ID
years = ['2005', '2010', '2015']
base = 'http://comtrade.un.org/api/get?'
url = '{}max=50000&type={}&freq={}&px={}'.format(
    base, prod_type, freq, classification)
df = pd.DataFrame(columns=['period', 'pt3ISO', 'rt3ISO', 'TradeValue'])

In [3]:
for n in chunker(c_codes.index.values[1:], 5):
    req = '&ps={}&r=all&p={}&rg=2&cc={}'.format(
        '%2C'.join(years), '%2C'.join(n), prod)
    r = requests.get('{}{}'.format(url, req)).json()['dataset']
    for f in r:
        df = df.append([f['period'], f['pt3ISO'], f['rt3ISO'], f['TradeValue']])
    time.sleep(5)

/home/brian/miniconda3/envs/bdcps/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/home/brian/miniconda3/envs/bdcps/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


ChunkedEncodingError: ('Connection broken: OSError("(104, \'ECONNRESET\')",)', OSError("(104, 'ECONNRESET')",))

In [ ]:
df.to_csv('440710.csv')